# Chicago TNP Data Exploration

Notes:

- Datasets to updated quarterly
- Starting and ending time rounded to nearest 15 minutes
- Trip fare rounded to nearest $2.50
- Tip rounded to nearest \$1.00

In [69]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import shape, Point
import folium

Quick Summary of Datasets:

In [7]:
d = {'dataset_name': ['Trips', 'Drivers', 'Vehicles'], 'total_obs': [17432011, 4812612, 4812049], 'date_reported_start': ['11/01/2018', '2015-02','2015-02'], 'date_reported_end': ['12/31/2018', '2018-12', '2018-12'], 'num_cols': [21, 7, 9]}
summary_df = pd.DataFrame(d)
summary_df

,dataset_name,total_obs,date_reported_start,date_reported_end,num_cols
0,Trips,17432011,11/01/2018,12/31/2018,21
1,Drivers,4812612,2015-02,2018-12,7
2,Vehicles,4812049,2015-02,2018-12,9


## Trips Dataset

In [ ]:
os = 0
params = {'$limit': 1000, '$offset': os}
response = requests.get('https://data.cityofchicago.org/resource/m6dm-c72p.json', params).json()
total_response = response
os = 1000
while len(total_response) <= 10000:
    params = {'$limit': 1000, '$offset': os}
    response = requests.get('https://data.cityofchicago.org/resource/m6dm-c72p.json', params).json()
    os += 1000
    total_response += response

In [ ]:
trips_df = pd.DataFrame.from_dict(total_response, dtype=str)

API to get first 1,000 (max to download w/o loop) from trips dataset & make pandas dataframe:

In [77]:
trips_response = requests.get('https://data.cityofchicago.org/resource/m6dm-c72p.json').json()
trips_df = pd.DataFrame.from_dict(trips_response, dtype=str)

In [78]:
trips_df.columns

Index(['additional_charges', 'dropoff_census_tract',
       'dropoff_centroid_latitude', 'dropoff_centroid_location',
       'dropoff_centroid_longitude', 'dropoff_community_area', 'fare',
       'pickup_census_tract', 'pickup_centroid_latitude',
       'pickup_centroid_location', 'pickup_centroid_longitude',
       'pickup_community_area', 'shared_trip_authorized', 'tip',
       'trip_end_timestamp', 'trip_id', 'trip_miles', 'trip_seconds',
       'trip_start_timestamp', 'trip_total', 'trips_pooled'],
      dtype='object')

In [79]:
trips_df.dtypes

additional_charges            object
dropoff_census_tract          object
dropoff_centroid_latitude     object
dropoff_centroid_location     object
dropoff_centroid_longitude    object
dropoff_community_area        object
fare                          object
pickup_census_tract           object
pickup_centroid_latitude      object
pickup_centroid_location      object
pickup_centroid_longitude     object
pickup_community_area         object
shared_trip_authorized        object
tip                           object
trip_end_timestamp            object
trip_id                       object
trip_miles                    object
trip_seconds                  object
trip_start_timestamp          object
trip_total                    object
trips_pooled                  object
dtype: object

See first five observations in dataset

In [80]:
trips_df.head(n=5)

,additional_charges,dropoff_census_tract,dropoff_centroid_latitude,dropoff_centroid_location,dropoff_centroid_longitude,dropoff_community_area,fare,pickup_census_tract,pickup_centroid_latitude,pickup_centroid_location,...,pickup_community_area,shared_trip_authorized,tip,trip_end_timestamp,trip_id,trip_miles,trip_seconds,trip_start_timestamp,trip_total,trips_pooled
0,0,NaN,41.9465114202,"{'type': 'Point', 'coordinates': [-87.80602000...",-87.8060200025,17,17.5,NaN,41.9012069941,"{'type': 'Point', 'coordinates': [-87.67635598...",...,24,True,3,2019-01-01T00:00:00.000,2d948e2b5abc7056145ecc026479462b00f7bcf0,10.87562822896,1613,2018-12-31T23:30:00.000,20.5,1
1,2.5,17031839000,41.8710158803,"{'type': 'Point', 'coordinates': [-87.63140652...",-87.6314065252,32,5,17031833100,41.8790669938,"{'type': 'Point', 'coordinates': [-87.65700502...",...,28,False,0,2019-01-01T00:00:00.000,fc4b6e92960f1f5bb1f3a2697e49b8913576e207,1.62441530144,562,2018-12-31T23:45:00.000,7.5,1
2,2.5,17031081403,41.8909220259,"{'type': 'Point', 'coordinates': [-87.61886835...",-87.6188683546,8,7.5,17031080300,41.9074919303,"{'type': 'Point', 'coordinates': [-87.63576009...",...,8,False,0,2019-01-01T00:00:00.000,c0f7e586f004df3987059c883705cb8b62e5cc6d,2.905180388,801,2018-12-31T23:45:00.000,10,1
3,7.5,NaN,41.968069,"{'type': 'Point', 'coordinates': [-87.72155906...",-87.7215590627,14,22.5,NaN,41.9802643146,"{'type': 'Point', 'coordinates': [-87.91362459...",...,76,False,0,2019-01-01T00:00:00.000,2a8d8287d0c7d5f86da6cc3189278d371c5ee2ef,11.0738640672,1223,2018-12-31T23:30:00.000,30,1
4,2.5,NaN,41.9227606205,"{'type': 'Point', 'coordinates': [-87.69915534...",-87.6991553432,22,12.5,NaN,41.899602111,"{'type': 'Point', 'coordinates': [-87.63330803...",...,8,False,0,2019-01-01T00:00:00.000,70a3da64b46c901b41078cc039bf4c36e6916baa,5.76561789088,1247,2018-12-31T23:30:00.000,15,1


Number of NaN/Null Values by Column

In [81]:
trips_df.isnull().sum(axis=0)

additional_charges              0
dropoff_census_tract          466
dropoff_centroid_latitude      37
dropoff_centroid_location      37
dropoff_centroid_longitude     37
dropoff_community_area         37
fare                            0
pickup_census_tract           465
pickup_centroid_latitude       42
pickup_centroid_location       42
pickup_centroid_longitude      42
pickup_community_area          42
shared_trip_authorized          0
tip                             0
trip_end_timestamp              0
trip_id                         0
trip_miles                      0
trip_seconds                    0
trip_start_timestamp            0
trip_total                      0
trips_pooled                    0
dtype: int64

For longest value in each column:

In [82]:
longest_val = 0
for column in trips_df.columns:
    for val in trips_df[column].unique():
        if len(str(val)) > longest_val:
            longest_val = len(str(val))
    print(column, longest_val)

additional_charges 5
dropoff_census_tract 11
dropoff_centroid_latitude 13
dropoff_centroid_location 70
dropoff_centroid_longitude 70
dropoff_community_area 70
fare 70
pickup_census_tract 70
pickup_centroid_latitude 70
pickup_centroid_location 70
pickup_centroid_longitude 70
pickup_community_area 70
shared_trip_authorized 70
tip 70
trip_end_timestamp 70
trip_id 70
trip_miles 70
trip_seconds 70
trip_start_timestamp 70
trip_total 70
trips_pooled 70


Converts timestamps from strings to datetime objects

In [83]:
trips_df['trip_start_timestamp'] = pd.to_datetime(trips_df['trip_start_timestamp'])
trips_df['trip_end_timestamp'] = pd.to_datetime(trips_df['trip_end_timestamp'])

In [107]:
trips_df.trip_start_timestamp.min()

Timestamp('2018-12-31 22:30:00')

In [109]:
trips_df.trip_start_timestamp.max()

Timestamp('2018-12-31 23:45:00')

In [108]:
trips_df.trip_end_timestamp.min()

Timestamp('2018-12-31 23:45:00')

In [110]:
trips_df.trip_end_timestamp.max()

Timestamp('2019-01-01 00:00:00')

Drop nulls to allow for mapping:

In [105]:
trips_df = trips_df[trips_df['pickup_centroid_latitude'].notnull()]
trips_df = trips_df[trips_df['dropoff_centroid_latitude'].notnull()]

In [106]:
xy = trips_df[['pickup_centroid_latitude', 'pickup_centroid_longitude']]
hmap = folium.Map(location=[41.87, -87.69], zoom_start=11)
hm_rod = HeatMap(list(zip(xy['pickup_centroid_latitude'].values, xy['pickup_centroid_longitude'].values)), radius=13, blur=20)
hmap.add_child(hm_rod)

In [104]:
xy = trips_df[['dropoff_centroid_latitude', 'dropoff_centroid_longitude']]
hmap = folium.Map(location=[41.87, -87.69], zoom_start=11)
hm_rod = HeatMap(list(zip(xy['dropoff_centroid_latitude'].values, xy['dropoff_centroid_longitude'].values)), radius=13, blur=20)
hmap.add_child(hm_rod)

## Driver Dataset

API to get first 1,000 (max to download w/o loop) from drivers dataset & make pandas dataframe:

In [55]:
dr_response = requests.get('https://data.cityofchicago.org/resource/j6wf-834c.json').json()
dr_df = pd.DataFrame.from_dict(dr_response)

In [56]:
dr_df.columns

Index(['city', 'driver_start_month', 'month_reported', 'multiple_tnps',
       'number_of_trips', 'state', 'zip'],
      dtype='object')

See first 5 observations in dataset

In [65]:
dr_df.head(n=5)

,city,driver_start_month,month_reported,multiple_tnps,number_of_trips,state,zip
0,Chicago,2014-09,2018-12,True,0,IL,60608
1,Antioch,2018-09,2018-12,False,16,IL,60002
2,Robbins,2017-11,2018-12,True,30,IL,60472
3,Schaumburg,2018-08,2018-12,True,193,IL,60193
4,Hammond,2018-05,2018-12,False,0,IN,46323


Number of NaN/Null Values by Column

In [66]:
dr_df.isnull().sum(axis=0)

city                  28
driver_start_month     0
month_reported         0
multiple_tnps          0
number_of_trips        0
state                  0
zip                   28
dtype: int64

For longest value in each column:

In [67]:
longest_val = 0
for column in dr_df.columns:
    for val in dr_df[column].unique():
        if len(str(val)) > longest_val:
            longest_val = len(str(val))
    print(column, longest_val)

city 18
driver_start_month 18
month_reported 18
multiple_tnps 18
number_of_trips 18
state 18
zip 18


# Vehicle Dataset

API to get first 1,000 (max to download w/o loop) from vehicles dataset & make pandas dataframe:

In [50]:
vehicle_response = requests.get('https://data.cityofchicago.org/resource/bc6b-sq4u.json').json()
vehicle_df = pd.DataFrame.from_dict(dr_response)

In [51]:
vehicle_df.columns

Index(['city', 'driver_start_month', 'month_reported', 'multiple_tnps',
       'number_of_trips', 'state', 'zip'],
      dtype='object')

See first 5 observations in dataset

In [52]:
vehicle_df.head(n=5)

,city,driver_start_month,month_reported,multiple_tnps,number_of_trips,state,zip
0,Chicago,2014-09,2018-12,True,0,IL,60608
1,Antioch,2018-09,2018-12,False,16,IL,60002
2,Robbins,2017-11,2018-12,True,30,IL,60472
3,Schaumburg,2018-08,2018-12,True,193,IL,60193
4,Hammond,2018-05,2018-12,False,0,IN,46323


Number of NaN/Null Values by Column

In [53]:
vehicle_df.isnull().sum(axis=0)

city                  28
driver_start_month     0
month_reported         0
multiple_tnps          0
number_of_trips        0
state                  0
zip                   28
dtype: int64

For longest value in each column:

In [54]:
longest_val = 0
for column in vehicle_df.columns:
    for val in vehicle_df[column].unique():
        if len(str(val)) > longest_val:
            longest_val = len(str(val))
    print(column, longest_val)

city 18
driver_start_month 18
month_reported 18
multiple_tnps 18
number_of_trips 18
state 18
zip 18
